In [1]:
%autosave 0

Autosave disabled


# html 显示结果的简单计算

## html-td后拼接字符串，需要getdata()返回数值型结果Decimal/int
- 若结果为None,返回0
    + if result is None: return 0
- getdata()返回数值，html拼接字符串，转成'''+str(result+result1)+'''计算后拼接

In [2]:
import pymysql
import time
import smtplib
from email.mime.text import MIMEText
from email.header import Header

In [3]:
def getdata(sql):
    conn = pymysql.connect(host='dev.rrx.mysql01.srv',
                          port=3306,
                          db='xcc',
                          user='dev',
                          passwd='KRkFcVCbopZbS8R7',
                          charset='utf8')
    cur = conn.cursor()
    try:
        cur.execute(sql)
        result = cur.fetchone()[0]
        return str(result)
    except Exception as e:
        raise e
    finally:
        conn.commit()
        cur.close()
        conn.close()

In [30]:
def getdata_01(sql):
    conn = pymysql.connect(host='dev.rrx.mysql01.srv',
                          port=3306,
                          db='xcc',
                          user='dev',
                          passwd='KRkFcVCbopZbS8R7',
                          charset='utf8')
    cur = conn.cursor()
    try:
        cur.execute(sql)
        result = cur.fetchone()[0]
        if result is None:
            return 0
        return result
    except Exception as e:
        raise e
    finally:
        conn.commit()
        cur.close()
        conn.close()

In [31]:
recharge_amount_sum = "select sum(amount) as'充值金额'\
                       from trade \
                       where trade_type=0\
                       or trade_type=1\
                       or trade_type=2\
                       and trade_status=1"
recharge_amount_sum_rst = getdata(recharge_amount_sum)

he_amount_sum = "select sum(amount) as'他查进账'\
                 from trade \
                 where trade_type=20\
                 or trade_type=21\
                 and trade_status=1"
he_amount_sum_rst = getdata(he_amount_sum)

withdraw_amount_sum = "select sum(amount) as'提现金额'\
                       from trade \
                       where trade_type=10\
                       and trade_status=1"
withdraw_amount_sum_rst = getdata(withdraw_amount_sum)


In [32]:
recharge_amount_sum = "select sum(amount) as'充值金额'\
                       from trade \
                       where trade_type=0\
                       or trade_type=1\
                       or trade_type=2\
                       and trade_status=1"
recharge_amount_sum_rst = getdata_01(recharge_amount_sum)

he_amount_sum = "select sum(amount) as'他查进账'\
                 from trade \
                 where trade_type=20\
                 or trade_type=21\
                 and trade_status=1"
he_amount_sum_rst = getdata_01(he_amount_sum)

withdraw_amount_sum = "select sum(amount) as'提现金额'\
                       from trade \
                       where trade_type=10\
                       and trade_status=1"
withdraw_amount_sum_rst = getdata_01(withdraw_amount_sum)


In [33]:
recharge_amount_sum_rst

Decimal('2130.53')

In [34]:
he_amount_sum_rst

Decimal('6.09')

In [35]:
type(withdraw_amount_sum_rst)

int

In [36]:
a = recharge_amount_sum_rst+he_amount_sum_rst+withdraw_amount_sum_rst

In [37]:
a

Decimal('2136.62')

In [19]:
# 发送邮箱服务器
smtpserver = "smtp.exmail.qq.com"

# 发送邮箱用户名密码
user = "qufang@jinjiedao.cn"
password = "Qf080501"

# 发送和接收邮箱
sender = "qufang@jinjiedao.cn"
receive = ["qufang@jinjiedao.cn"]

ctime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
# 发送邮件主题和内容
subject = "险查查数据统计监测_数据部门_"+ctime+""

In [38]:
content = '''<!DOCTYPE html>
<html lang='en'>
<head>
    <meta charset="UTF-8">
    <title>D1风控数据统计监测</title>
    <meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no">
    <style type="text/css">
        table {
            border-collapse:collapse;
        }
        table, td, th {
            border: 1px solid black;
            text-align: center;
        }
    </style>
</head>
<body>
<table border="1" cellspacing="0" align="center">
    <tr>
        <td>充值金额</td>
        <td>'''+str(recharge_amount_sum_rst)+'''</td>
    </tr>
    <tr>
        <td>他查进账</td>
        <td>'''+str(he_amount_sum_rst)+'''</td>
    </tr>
    <tr>
        <td>提现金额</td>
        <td>'''+str(withdraw_amount_sum_rst)+'''</td>
    </tr>
    <tr>
        <td>数据汇总</td>
        <td>'''+str(recharge_amount_sum_rst+he_amount_sum_rst+withdraw_amount_sum_rst)+'''</td>
    </tr>
</table>
</body>
</html>'''

In [39]:
# HTML邮件正文
msg = MIMEText(content, 'html', 'utf-8')
msg['Subject'] = Header(subject, 'utf-8')
msg['From'] = "qufang@jinjiedao.cn"
msg['To'] = ','.join(receive)


# SSL协议端口号要使用465
smtp = smtplib.SMTP_SSL(smtpserver, 465)

# HELO向服务器标志用户身份
smtp.helo(smtpserver)

# 服务器返回结果确认
smtp.ehlo(smtpserver)

# 登录邮箱服务器用户名密码
smtp.login(user, password)

print("Send email start...")
smtp.sendmail(sender, receive, msg.as_string())
smtp.quit()
print("email send end!")

Send email start...
email send end!
